# Import

In [89]:
import pandas as pd
import pm4py as pm
import numpy as np
import psutil as pu

import matplotlib.pyplot as plt
import seaborn as sns

import datetime

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score  

from collections import defaultdict

training_file_path = 'db/BPI_Challenge_2012.XE-training.csv'
testing_file_path = 'db/BPI_Challenge_2012.XE-test.csv'

# load CSV with pandas and pm4py

pm4py gives two extra columns: @@index and @@case_index

In [90]:
def format_event_log(path):
    event_log = pd.read_csv(path, sep=',')
    event_log = pm.format_dataframe(
        event_log, 
        case_id='case concept:name', 
        activity_key='event concept:name', 
        timestamp_key='event time:timestamp'
    )
    event_log = event_log.drop("case:concept:name", axis = 1)
    event_log = event_log.drop("concept:name", axis = 1)
    event_log = event_log.drop("time:timestamp", axis = 1)
    return event_log
#event_log

# Helper methods

## Genernal

Function that finds the length of the longest trace in a list of traces

In [91]:
def find_longest_trace(traces):
    max = 0
    for trace in traces:
        if trace.shape[0] > max:
            max = trace.shape[0]
    return max

def get_all_Activities():
    df = format_event_log(training_file_path)
    arr = df['event concept:name'].unique()
    return arr

## Sprint 1

Function that adds the Ground truth columns for event and time of the event
- Every value in the ground truth columns is the actual next event or time of the next evet in the data set
- Used for naive simulator
- Used for verification purposes

In [92]:
# assume sorted by caseID and time
def caseHasNextEvent(df, index):
    if index >= len(df) - 1:
        return False
    if df.loc[index, 'case concept:name'] == df.loc[index+1, 'case concept:name']:
        return True
    return False


def computeTimeDifference(df):
    # df['time_until_next_event'] = 0  # initialize new column with zeros
    df = df.assign(ground_truth_time=0) # initialize new column

    # iterate over each row of the dataframe
    for i, row in df.iterrows():        
        # check if there is a next row with the same case
        if caseHasNextEvent(df, i):
            nextTime = df.loc[i+1, 'event time:timestamp']
            currentTime = row['event time:timestamp']
            timeDiff = nextTime - currentTime
            df.at[i, 'ground_truth_time'] = timeDiff.total_seconds()
    
    return df



In [93]:

def writeGroundtruth(df):
    df = df.sort_values(by=['case concept:name','event time:timestamp'])
    #add new columns containing the name of the next event in the case and the time when it happens
    df = df.assign(ground_truth_activity='')
    df = df.assign(ground_truth_time='')

    for ind in df.index:
        if caseHasNextEvent(df, ind):
            df.at[ind,'ground_truth_activity'] = df.loc[ind+1,'event concept:name']
            #df.at[ind,'ground_truth_time'] = df.loc[ind+1, 'event time:timestamp']
        else:
            df.at[ind,'ground_truth_activity'] = None
            #df.at[ind,'ground_truth_time'] = None
    df = computeTimeDifference(df)
    return df

#df_event = writeGroundtruth(event_log)
#df_event.to_csv('check_writeGroundtruth_out.csv')

## Sprint 2

Function that splits the data set into separate traces
- Parameter $traces$ is a list containing all the individual traces
- Each trace in traces is a list containing all the events in the trace in the order that they happen
- Helper method for prefix extraction

In [94]:
def split_into_traces(event_log):
    traces = []
    last_location = 0
    for j in range (0, event_log.shape[0] - 1):
        if not event_log["case concept:name"][j] == event_log["case concept:name"][j + 1]:
            traces.append(event_log.loc[last_location : j].reset_index())
            last_location = j + 1
    traces.append(event_log.loc[last_location : event_log.shape[0] - 1].reset_index())
    return traces

Function that deletes traces from an event log that do not end before a specified time

In [95]:
def delete_traces(event_log, end_time):
    traces = split_into_traces(event_log)
    del_tr = []
    for j in range(len(traces)):
        for k in traces[j].index:
            trace_ts = traces[j]["event time:timestamp"][k]
        if trace_ts > end_time:
            del_tr.append(traces[j]["case concept:name"][0])
    
    for j in range(len(del_tr)):
        event_log = event_log[event_log["case concept:name"] != del_tr[j]]
    event_log.to_csv("db/S2_Remove_Overlap.csv")
    return event_log

Prefix extraction
- extract event lists of odd length like 1, 3, 5, 7, 9, 11, 13, 15 from the traces for prediction
- and the ground truth
- store them in a list of prefixes

In [96]:
def prefix_extraction(traces, prefix_lengths):
    prefixes = []
    for trace in traces:
        for length in prefix_lengths:
            if trace.shape[0] >= length:
                prefixes.append(trace.loc[:length - 1])
    return prefixes

Regression helper methods

In [97]:
# dict_independent_variable should have index as activities
# and with a value of a discrete number.
def process_X_Y(df, str_DV_col, dict_independent_variable):
    independent_Variable = dict_independent_variable.keys()
    dependent_Variable = str_DV_col
    # independent_Variable = get_all_Activities().tolist()

    # dict_activity_to_int = {a: independent_Variable.index(a) for a in independent_Variable}
    
    X = df[independent_Variable]
    if str_DV_col == "ground_truth_activity":
        Y = df[dependent_Variable].replace(dict_independent_variable)
    elif str_DV_col == "ground_truth_time":
        Y = df[dependent_Variable]

    print(independent_Variable)
    return X, Y

def get_Regression_model_and_scaler(X, Y, type):

    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    if type == "Logistic":
        model = LogisticRegression()
        Y = Y.fillna(0.0)
    elif type == "Linear":
        model = LinearRegression()
        Y = Y.fillna(0.0)


    print(Y)
    model.fit(X, Y)
    
    intercept = model.intercept_

    return model, scaler


## Sprint 3

isHolliday helper method

In [98]:
def isHoliday(timestamp):

    day_of_week = timestamp.weekday()
    
    if day_of_week in [5, 6]:
        return True
    else:
        return False

## Aggregation functions

Aggregation encoding

- encode traces into numerical data for prediction
- adds the ground truth at the end of the encoded prefix

In [99]:
def create_dict(events):

    events_dict = {'ground_truth_activity' : 'truth', 'ground_truth_time' : 'truth'}
    for event in events:
        events_dict[event] = 0
    events_dict['hollidays'] = 0
    return events_dict

def aggregation_encoding(prefixes, event_log):
    events = get_all_Activities()
    event_dict = create_dict(events)
    aggregation_encoding = pd.DataFrame(event_dict, index=[0])
    aggregation_encoding = aggregation_encoding.drop(0)
    #new_encoding = ['truth', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    index = 0
    for prefix in prefixes:
        current_dict = create_dict(events)
        current_dict['ground_truth_activity'] = prefix['ground_truth_activity'][len(prefix.index) - 1]
        current_dict['ground_truth_time'] = prefix['ground_truth_time'][len(prefix.index) - 1]
        for j in range (prefix.shape[0]):
            if isHoliday(prefix["event time:timestamp"][j]):
                current_dict['hollidays'] = current_dict['hollidays'] + 1
            current_dict[prefix["event concept:name"][j]] = current_dict[prefix["event concept:name"][j]] + 1
        aggregation_encoding.loc[index] = current_dict.values()
        index += 1
    return aggregation_encoding

In [100]:
def get_aggregation_encoding(path, type):
    event_log = format_event_log(path)
    if type == "train":
        time_split = pd.to_datetime("02-01-2012 15:28:39.244",format="%d-%m-%Y %H:%M:%S.%f")
        #event_log = delete_traces(event_log, pd.Timestamp(time_split, tz='GMT+0'))
        #event_log = format_event_log("db/S2_Remove_Overlap.csv")
        print("done remove")
    traces = split_into_traces(event_log)
    print("done traces")
    prefix_lengths = []
    if type == "train":
        prefix_lengths = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29]
    if type == "test":
        prefix_lengths = [x for x in range(1, find_longest_trace(traces) + 1)]
    prefixes = prefix_extraction(traces, prefix_lengths)
    print("done prefixes")
    aggregation = aggregation_encoding(prefixes, event_log)
    if type == "train":
        aggregation.to_csv("db/aggregation_encoding_train.csv")
    if type == "test":
        aggregation.to_csv("db/aggregation_encoding_test.csv")
    return aggregation
     

In [101]:
def write_aggregation():
    train_file_path = 'db/S1_GroundTruth.csv'
    test_file_path = 'db/S2_Aggregation_Estimator.csv'
    train_aggregation = get_aggregation_encoding(train_file_path, "train")
    test_aggregation = get_aggregation_encoding(test_file_path, "test")

# Naive Estimator (S1)
- used indiations on canvas

In [102]:
def naive_estimators():
    df_train = format_event_log(training_file_path)
    df_train = writeGroundtruth(df_train)

    df_test = format_event_log(testing_file_path)
    df_test = writeGroundtruth(df_test)

    traces = split_into_traces(df_train)
    N = find_longest_trace(traces)
    
    common_activities = []
    common_times = []
    for i in range (0, N):
        activity_at_position_i = defaultdict(int)
        for trace in traces:
            if (i < trace.shape[0]):
                activity_at_position_i[trace["event concept:name"][i]] += 1
    
        common_activities.append(max(activity_at_position_i.keys()))

        time_at_position_i = defaultdict(int)
        for trace in traces:
            if (i < trace.shape[0]):
                time_at_position_i[trace["ground_truth_time"][i]] += 1
    
        common_times.append(max(time_at_position_i.keys()))

    activity_predictions = []
    time_predictions = []
    index = 0
    for j in range (0, df_test.shape[0] - 1):
        if df_test["case concept:name"][j] == df_test["case concept:name"][j + 1]:
            activity_predictions.append(common_activities[index])
            time_predictions.append(common_times[index])
            index += 1
        else:
            activity_predictions.append(common_activities[index])
            time_predictions.append(common_times[index])
            index = 0
    activity_predictions.append(common_activities[index])
    time_predictions.append(common_times[index])
    df_test["S_1_Activity"] = activity_predictions
    df_test["S_1_Time"] = time_predictions
    df_train.to_csv("db/S1_GroundTruth.csv")
    df_test.to_csv("db/S1_Naive_estimator.csv")


# Mode Estimator (S2.1)
1. for each row find next activity of the case and its timestamp
2. compute the time it take for the next event to be log in the db
3. for each activity find the most common next activity (mode)
4. for each activity find the average time between next activity
5. Have 3. and 4. in a DataFrame
6. base on the current activity write the prediction of the next activity and time it will take

In [103]:
def mode_estimators():

    s2_train_file_path = "db/S1_GroundTruth.csv"
    s2_test_file_path = "db/S1_Naive_estimator.csv"

    df_train = format_event_log(s2_train_file_path)

    df_test = format_event_log(s2_test_file_path)


    df_naive_predictor_dict = df_train.groupby(['event concept:name']).agg(
        S_2_Activity = ('ground_truth_activity', pd.Series.mode),
        S_2_Time = ('ground_truth_time', 'mean')
    )

    df_test = df_test.assign(S_2_Activity='')
    df_test = df_test.assign(S_2_Time=0)
    for i, r in df_test.iterrows():
        this_event = r['event concept:name']
        next_event = df_naive_predictor_dict.loc[this_event,'S_2_Activity']
        next_event_time = df_naive_predictor_dict.loc[this_event,'S_2_Time']
        df_test.at[i,'S_2_Activity'] = next_event
        df_test.at[i,'S_2_Time'] = next_event_time
    df_test.to_csv('db/S2_Aggregation_Estimator.csv')
    


# Multivalue Regression (S2.2)

## Activity Estimator

In [104]:
def regression_estimator():
    df_encoded_train = pd.read_csv('db/aggregation_encoding_train.csv')
    df_encoded_test = pd.read_csv('db/aggregation_encoding_test.csv')
    
    df_output = format_event_log('db/S2_Aggregation_Estimator.csv')
    
    independent_Variable = get_all_Activities().tolist()
    dict_IV_index_activities = {
        a: independent_Variable.index(a) for a in independent_Variable
    }
    dict_IV_invert = {v: k for k, v in dict_IV_index_activities.items()}
    # ========================================================================================== 
    X_train_activity, Y_train_activity = process_X_Y(df_encoded_train, "ground_truth_activity", dict_IV_index_activities)
    X_test_activity, Y_test_activity = process_X_Y(df_encoded_test, "ground_truth_activity", dict_IV_index_activities)

    model_activity, scaler_activity = get_Regression_model_and_scaler(
        X_train_activity, 
        Y_train_activity, 
        "Logistic"
    )

    X_test_activity = scaler_activity.transform(X_test_activity)

    predicts_activity = model_activity.predict(X_test_activity)
    predicts_activity = [dict_IV_invert[p] for p in predicts_activity]

    # ===========================================================================================
    X_train_time, Y_train_time = process_X_Y(df_encoded_train, "ground_truth_time", dict_IV_index_activities)
    X_test_time, Y_test_time = process_X_Y(df_encoded_test, "ground_truth_time", dict_IV_index_activities)
    model_time, scaler_time = get_Regression_model_and_scaler(
        X_train_time,
        Y_train_time,
        "Linear"
    )
    X_test_time = scaler_time.transform(X_test_time)
    predicts_time = model_time.predict(X_test_time)

    df_output['S_3_activities'] = predicts_activity
    df_output['S_3_time'] = predicts_time
    df_output.to_csv("db/S3_Regression_Estimator.csv")



# Other Functionalities

## Visualisation

Scatterplot representation of the entire data set in chronological order

In [105]:
def plot_all_data():
    df_BPI_train = pd.read_csv(training_file_path)
    df_BPI_test = pd.read_csv(testing_file_path)
    df_BPI = pd.concat([df_BPI_train, df_BPI_train])
    
    df_BPI['event time:timestamp'] = pd.to_datetime(df_BPI['event time:timestamp'],format="%d-%m-%Y %H:%M:%S.%f")
    
    ax = sns.relplot(data=df_BPI, x='event time:timestamp', y='case concept:name', hue='event concept:name', height=10, aspect=1.5)
    plt.ylim(max(df_BPI['case concept:name']), min(df_BPI['case concept:name']))
    plt.show()

Scatterplot representation of the data set split into training and test sets, after the removal of unusable traces from the training set

In [106]:
def plot_split_data():
    df_BPI_train = pd.read_csv(training_file_path)
    df_BPI_test = pd.read_csv(testing_file_path)
    df_BPI = pd.concat([df_BPI_train, df_BPI_train])
    
    df_BPI['event time:timestamp'] = pd.to_datetime(df_BPI['event time:timestamp'],format="%d-%m-%Y %H:%M:%S.%f")
    df_BPI['case REG_DATE'] = pd.to_datetime(df_BPI['case REG_DATE'],format="%d-%m-%Y %H:%M:%S.%f")
    splittime = pd.to_datetime("02-01-2012 15:28:39.244",format="%d-%m-%Y %H:%M:%S.%f")
    plt_splitline = df_BPI.copy()
    plt_splitline['event time:timestamp'] = splittime
    plt_splitline['event concept:name'] = "test/training split"
    
    df_BPI = df_BPI.sort_values(by=['case concept:name','event time:timestamp'], ignore_index=True)
    df_BPI_train = delete_traces(df_BPI.copy(), splittime)
    df_BPI_test = df_BPI[df_BPI['case REG_DATE'] >= splittime]
    
    df_BPI = pd.concat([df_BPI_train, df_BPI_test])
    df_BPI = df_BPI.sort_values(by='event time:timestamp', ignore_index=True)
    df_BPI = pd.concat([df_BPI, plt_splitline])
    
    ax = sns.relplot(data=df_BPI, x='event time:timestamp', y='case concept:name', hue='event concept:name', height=10, aspect=1.5)
    plt.ylim(max(df_BPI['case concept:name']), min(df_BPI['case concept:name']))
    plt.show()

Scatterplot representation of the data set with time arrangement of events relative to the start of their respective case

In [107]:
def plot_case_relative():
    df_BPI_train = pd.read_csv(training_file_path)
    df_BPI_test = pd.read_csv(testing_file_path)
    df_BPI = pd.concat([df_BPI_train, df_BPI_train])
    
    df_BPI['event time:timestamp'] = pd.to_datetime(df_BPI['event time:timestamp'],format="%d-%m-%Y %H:%M:%S.%f")
    df_BPI['case REG_DATE'] = pd.to_datetime(df_BPI['case REG_DATE'],format="%d-%m-%Y %H:%M:%S.%f")
    
    df_BPI['event time:case relative'] = pd.to_datetime(df_BPI['event time:timestamp'], format="%Y-%m-%d %H:%M:%S.%f") - pd.to_datetime(df_BPI['case REG_DATE'], format="%Y-%m-%d %H:%M:%S.%f")
    
    ax = sns.relplot(data=df_BPI, x='event time:case relative', y='case concept:name', hue='event concept:name', markers="x", height=7, aspect=1.5)
    plt.ylim(max(df_BPI['case concept:name']), min(df_BPI['case concept:name']))
    plt.xlim(0, )
    plt.show()

## Error model

In [108]:
def error_model(predictor_path, prediction_activity, prediction_time):
    df_error_model = pd.read_csv(predictor_path)
    df_error_model['error_activity'] = df_error_model['ground_truth_activity'] == df_error_model[prediction_activity]
    df_error_model['error_time'] = df_error_model[prediction_time] - df_error_model['ground_truth_time']
    correct_predictions = df_error_model['error_activity'].value_counts()
    percentage = correct_predictions[True] / len(df_error_model['error_activity']) * 100
    mean_absolute_error = df_error_model['error_time'].mean()
    print(f'Percentage of True values: {percentage:.1f}%')
    print(mean_absolute_error)

## Memory and CPU Usage

In [109]:
def check_memory_cpu() : 
    print('The CPU usage is: ', pu.cpu_percent(4))
    print('RAM memory % used:', pu.virtual_memory()[2])
    print('RAM Used (GB):', pu.virtual_memory()[3]/1000000000)

# Main class
1. All functions that are needed all called in this cell.
2. No other cell runs code other than the main cell.

In [110]:
#naive_estimators()
#mode_estimators()
write_aggregation()
#regression_estimator()

<ipython-input-90-0b8fafbf99eb>:3: DeprecatedWarning: format_dataframe is deprecated as of 2.3.0 and will be removed in 3.0.0. the format_dataframe function does not need application anymore.
  event_log = pm.format_dataframe(


done remove
done traces
done prefixes


<ipython-input-90-0b8fafbf99eb>:3: DeprecatedWarning: format_dataframe is deprecated as of 2.3.0 and will be removed in 3.0.0. the format_dataframe function does not need application anymore.
  event_log = pm.format_dataframe(
<ipython-input-90-0b8fafbf99eb>:3: DeprecatedWarning: format_dataframe is deprecated as of 2.3.0 and will be removed in 3.0.0. the format_dataframe function does not need application anymore.
  event_log = pm.format_dataframe(


done traces
done prefixes


<ipython-input-90-0b8fafbf99eb>:3: DeprecatedWarning: format_dataframe is deprecated as of 2.3.0 and will be removed in 3.0.0. the format_dataframe function does not need application anymore.
  event_log = pm.format_dataframe(
